In [4]:
import os
import compiam

## 1. Accessing the Saraga Audiovisual Dataset  

>Saraga Audiovisual is a dataset that includes diverse renditions of **Carnatic vocal performances**, totaling **42 concerts** and more than **60 hours of music** featuring **Video recordings** for all concerts, enabling a wide range of multimodal analyses and **High-quality human pose estimation data** of musicians.

## Download and Extract Saraga Dataset

The Saraga dataset is available on Zenodo. You can download it using the following link:  
[Zondo Link](https://zenodo.org/records/15102483)  

The dataset is split into multiple parts, each containing specific components:

- **`saraga_audio.zip`** – Multi-track audio files along with their corresponding mixture files.
- **`saraga_gesture.zip`** – Pose estimation files extracted from videos corresponding to each audio track.
- **`saraga_metadata.zip`** – Metadata for all the audio files.
- **`saraga_video.zip`** – Videos from three sample concerts. Due to size constraints, only these three concerts are included. For access to the full video collection, contact the dataset providers.

### Step 1: Download the Dataset

Visit [Zenodo](https://zenodo.org/records/15102483) and manually download the required zip files.

Alternatively, you can use `wget` to download the files directly.

In [ ]:
!wget -O saraga_audio.zip "https://zenodo.org/records/15102483/files/saraga_audio.zip"
!wget -O saraga_gesture.zip "https://zenodo.org/records/15102483/files/saraga_gesture.zip"
!wget -O saraga_metadata.zip "https://zenodo.org/records/15102483/files/saraga_metadata.zip"
!wget -O saraga_video.zip "https://zenodo.org/records/15102483/files/saraga_video.zip"

### Step 2: Extract the Dataset

Once the files are downloaded, extract them into a common folder.
For that we use `zipfile`, python liibrary to deal zip files.

In [8]:
import zipfile

In [10]:
extract_path = "./saraga_audiovisual"
zip_files = ["saraga_audio.zip", "saraga_gesture.zip", "saraga_metadata.zip", "saraga_video.zip"]

In [ ]:
for zip_file in zip_files:
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"Extracted {zip_file} to {extract_path}")